In [1]:
from models import ast_models
from Dataset.dataset import load_LJSpeech

import torch
import torch.nn.functional as F
import torch.nn as nn
import torchaudio

import numpy as np
import random
import time

In [3]:
train_loader = load_LJSpeech('Dataset/pitch_scaling2.txt', speaker_emb=True)

In [4]:
len(train_loader)

3483

In [2]:
input_tdim = 500
ast_mdl = ast_models.ASTModel(label_dim=1,
                    fshape=128, tshape=2, fstride=128, tstride=1,
                    input_fdim=128, input_tdim=input_tdim, model_size='base',
                    pretrain_stage=False, load_pretrained_mdl_path='./save_model/SSAST-Base-Frame-400.pth')
                      

ast_mdl = ast_mdl.cuda()

now load a SSL pretrained models from ./save_model/SSAST-Base-Frame-400.pth
pretraining patch split stride: frequency=128, time=2
pretraining patch shape: frequency=128, time=2
pretraining patch array dimension: frequency=1, time=512
pretraining number of patches=512
fine-tuning patch split stride: frequncey=128, time=1
fine-tuning number of patches=499


D:\anaconda\envs\pytorch19\lib\site-packages\torch\nn\functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [6]:
learning_rate = 1e-05
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(ast_mdl.parameters(), lr = learning_rate, betas=[0.5, 0.999])

In [7]:
def train_spk(num_eps):
    for eps in range(num_eps):
        i = 0
        ast_mdl.train()
        since = time.time()
        for i, (fbank, embedding, label_freq, label_time) in enumerate(train_loader): #fbank: [B, D, T];  
            fbank = fbank.cuda() #确保embedding的shape:[B, 192]
            embedding = embedding.cuda()
            label_freq = label_freq.cuda()
            label_time = label_time.cuda()
            
            # #warming up
            # if global_step <= 1000 and global_step % 50 == 0 and args.warmup == True:
            #     for group_id, param_group in enumerate(optimizer.param_groups):
            #         warm_lr = (global_step / 1000) * lr_list[group_id]
            #         param_group['lr'] = warm_lr
            #         print('warm-up learning rate is {:f}'.format(param_group['lr']))                 
                    
            pred_freq, pred_time = ast_mdl(fbank, x_vector=embedding, task='ft_spk') #
            
            loss_freq = loss_fn(pred_freq.float(), label_freq.float())
            loss_time = loss_fn(pred_time.float(), label_time.float())
            loss = 1*loss_freq + 1*loss_time #set two hyperparemeters
            opt.zero_grad()
            loss.backward()
            opt.step()

            if i % 32 == 0:
                print ('.', end='')
            i += 1
            
            
        time_elapsed = time.time() - since    
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        if eps % 1 == 0:    
            print('loss:', loss, '\n' 'loss_freq:', loss_freq, '\n' 'loss_time:',loss_time)


        if (eps + 1) % 5 == 0:
            torch.save(ast_mdl.state_dict(), 'save_model/ft_spk2'+str(eps)+'.pth')
            print ('Saving checkpoint for epoch {} at {}'.format(eps,
                                            'save_model/ft_spk2'+str(eps)+'.pth'))

In [9]:
train_spk(100)

.............................................................................................................Training complete in 36m 1s
loss: tensor(0.0275, device='cuda:0', grad_fn=<AddBackward0>) 
loss_freq: tensor(0.0256, device='cuda:0', grad_fn=<MseLossBackward0>) 
loss_time: tensor(0.0019, device='cuda:0', grad_fn=<MseLossBackward0>)
.............................................................................................................Training complete in 35m 51s
loss: tensor(0.0317, device='cuda:0', grad_fn=<AddBackward0>) 
loss_freq: tensor(0.0301, device='cuda:0', grad_fn=<MseLossBackward0>) 
loss_time: tensor(0.0016, device='cuda:0', grad_fn=<MseLossBackward0>)
.............................................................................................................Training complete in 35m 57s
loss: tensor(0.0312, device='cuda:0', grad_fn=<AddBackward0>) 
loss_freq: tensor(0.0287, device='cuda:0', grad_fn=<MseLossBackward0>) 
loss_time: tensor(0.0025, device='cuda:0

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\envs\pytorch19\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LYC\AppData\Local\Temp/ipykernel_7488/3894772195.py", line 1, in <module>
    train_spk(100)
  File "C:\Users\LYC\AppData\Local\Temp/ipykernel_7488/941379688.py", line 26, in train_spk
    opt.step()
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\optim\optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\autograd\grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\optim\adam.py", line 133, in step
    F.adam(params_with_grad,
  File "D:\anaconda\envs\pytorch19\lib\site-packages\torch\optim\_functional.py", line 98, in adam
    param.addcdiv_(exp_avg, denom, value=-step_size)
KeyboardInterrupt

During handl

TypeError: object of type 'NoneType' has no len()

In [12]:
eps = 15
torch.save(ast_mdl.state_dict(), 'save_model/ft_spk'+str(eps)+'.pth')
print ('Saving checkpoint for epoch {} at {}'.format(eps,
                                            'save_model/ft_spk'+str(eps)+'.pth'))

Saving checkpoint for epoch 15 at save_model/ft_spk15.pth


##### Valid

In [3]:
def get_mel(audio, sr):
    audio_norm = audio / 32768.0
    audio_norm = audio_norm.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    # melspec = self.stft(sr).mel_spectrogram(audio_norm)
    melspec = torchaudio.compliance.kaldi.fbank(audio, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                    window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10)
         
    melspec = torch.squeeze(melspec, 0)
    # melspec = torch.transpose(melspec, 0, 1)
        
    #cut and pad
    target_length = 500
    n_frames = melspec.shape[0]
    p = target_length - n_frames

    if p > 0:
        m = torch.nn.ZeroPad2d((0, 0, 0, p))
        melspec = m(melspec)
    elif p < 0:
        melspec = melspec[0:target_length, :]
   
    return melspec #(T, D)

#### eval 2

In [4]:
state_dict = torch.load('./save_model/ft_spk24.pth')
ast_mdl.load_state_dict(state_dict, strict=False)
ast_mdl.train()

ASTModel(
  (v): DistilledVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(128, 2), stride=(128, 1))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((768

In [5]:
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
D:\anaconda\envs\pytorch19\lib\site-packages\huggingface_hub\utils\_deprecation.py:39: FutureWarning: Pass library_name=False as keyword args. From version 0.7 passing these as positional arguments will result in an error
  warnings.warn(


In [22]:
audio_org, sr_org =torchaudio.load('E:/datasets/project3/AISHELL-3/train/wav/SSB0018/SSB00180007.wav')
audio, sr =torchaudio.load('E:/datasets/project3/AISHELL-3/scaled/SSB0073/SSB00730025_fn7_t0.6000000000000001.wav')
mel = get_mel(audio, sr).unsqueeze(0).cuda()
emb = classifier.encode_batch(audio_org).squeeze(0).cuda()

In [23]:
print(mel.shape, emb.shape)

torch.Size([1, 500, 128]) torch.Size([1, 192])


In [24]:
pred_freq, pred_time = ast_mdl(mel, x_vector=emb, task='ft_spk')

In [25]:
print(pred_freq, pred_time)

tensor([[-6.9591]], device='cuda:0', grad_fn=<MulBackward0>) tensor([[0.5498]], device='cuda:0', grad_fn=<MulBackward0>)


##### calculate

In [23]:
import numpy as np
12*np.log2(0.6)

-8.843587129994475

In [123]:
1.58-1.12

0.45999999999999996

In [22]:
1/1.6

0.625

In [ ]:
1.62

In [108]:
1/1.9483

0.5132679772109018

In [109]:
-10.928821+3.3762

-7.552620999999999

In [103]:
-10.2898+4.6135

-5.6762999999999995

In [ ]:
6.7532,6.8445,6.688,6.7,-4.2455,

#### eval 1

In [139]:
state_dict = torch.load('./save_model/ft15.pth')
ast_mdl.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['speaker_embedding.0.weight', 'speaker_embedding.0.bias', 'speaker_embedding.1.weight', 'speaker_embedding.1.bias'], unexpected_keys=[])

In [144]:
pred_freq, pred_time = ast_mdl(mel, task='ft_cls2')

In [145]:
print(pred_freq, pred_time)

tensor([[1.0884]], device='cuda:0', grad_fn=<MulBackward0>) tensor([[1.1221]], device='cuda:0', grad_fn=<MulBackward0>)
